In [1]:
import time
start_time = time.time()

import pandas as pd

In [2]:
MMSTA_path = "../Data/Raw/MMSTA UPLOAD DHN BENITA LOWDASH.XLSX"

In [3]:
df_mmsta = pd.read_excel(MMSTA_path)
df_mmsta

,Level,Plant,Material Type,Material,Material Description,Bill of material,Plant-sp.matl status,Plant-sp.matl status.1,BOM status,BOM status.1,...,MRP profile,MRP profile.1,Lot size,Minimum Lot Size,Rounding value,Spare part indicator,Item Category,Component unit,Component quantity,Status
0,0,MA13,YFG,F000292917,240105971V,6760379.0,2,NaN,1,0,...,A111,NaN,EX,0,0,NaN,L,NaN,0,NaN
1,*1,MA13,YSFG,S008548980,Super Group 020 See drawing,6367154.0,2,NaN,1,0,...,A210,NaN,YX,0,25,NaN,L,PC,1,NaN
2,**2,MA13,YSFG,S008548890,Twisted Wire 26 ZZ,6367066.0,2,NaN,1,0,...,A210,NaN,YX,0,25,NaN,L,PC,1,NaN
3,***3,MA13,YSFG,S008548501,Circuit 1040 COFLRYB 0.35 L/W,6366678.0,92,NaN,92,0,...,A221,NaN,YX,0,25,M,L,PC,1,NaN
4,****4,MA13,YDM,FW406994,LT Single Wire COFLRYB 0.35 L/W,NaN,92,NaN,0,0,...,A305,NaN,ZK,0,14000,NaN,L,MM,1595,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117747,*1,MA13,YDM,430211570,PVC/Vinyl Tape 15mm Y,NaN,2,NaN,0,0,...,A305,NaN,ZK,0,3960,NaN,L,MM,332,NaN
117748,*1,MA13,YDM,430AM4530,Textile/Cloth Tape 45mm B,NaN,2,NaN,0,0,...,A305,NaN,ZK,0,540,NaN,L,MM,4163,NaN
117749,*1,MA13,YDM,430FF1930,PVC/Vinyl Tape 19mm B,NaN,2,NaN,0,0,...,A305,NaN,ZK,0,3168,NaN,L,MM,84914,NaN
117750,*1,MA13,YDM,430H31930,Textile/Cloth Tape 19mm B,NaN,2,NaN,0,0,...,A305,NaN,ZK,0,3600,NaN,L,MM,26693,NaN


In [4]:
df_mmsta.columns

Index(['Level', 'Plant', 'Material Type', 'Material', 'Material Description',
       'Bill of material', 'Plant-sp.matl status', 'Plant-sp.matl status.1',
       'BOM status', 'BOM status.1', 'Change Number', 'Changed on',
       'MRP Controller', 'MRP Controller.1', 'MRP profile', 'MRP profile.1',
       'Lot size', 'Minimum Lot Size', 'Rounding value',
       'Spare part indicator', 'Item Category', 'Component unit',
       'Component quantity', 'Status'],
      dtype='object')

In [5]:
df_mmsta = df_mmsta[['Level', 'Material Type' , 'Material', 'Material Description', 'Component quantity', 'Status']]

In [6]:
def add_columns(df):
    cols_to_add = ['PN','SN1','DES1', 'SN2','DES2', 'SN3','DES3']
    col_index = df.columns.get_loc('Material Description') + 1
    for i, col in enumerate(cols_to_add):
        df.insert(col_index + i, col, '')
    return df
df_mmsta = add_columns(df_mmsta)
df_mmsta

,Level,Material Type,Material,Material Description,PN,SN1,DES1,SN2,DES2,SN3,DES3,Component quantity,Status
0,0,YFG,F000292917,240105971V,,,,,,,,0,NaN
1,*1,YSFG,S008548980,Super Group 020 See drawing,,,,,,,,1,NaN
2,**2,YSFG,S008548890,Twisted Wire 26 ZZ,,,,,,,,1,NaN
3,***3,YSFG,S008548501,Circuit 1040 COFLRYB 0.35 L/W,,,,,,,,1,NaN
4,****4,YDM,FW406994,LT Single Wire COFLRYB 0.35 L/W,,,,,,,,1595,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
117747,*1,YDM,430211570,PVC/Vinyl Tape 15mm Y,,,,,,,,332,NaN
117748,*1,YDM,430AM4530,Textile/Cloth Tape 45mm B,,,,,,,,4163,NaN
117749,*1,YDM,430FF1930,PVC/Vinyl Tape 19mm B,,,,,,,,84914,NaN
117750,*1,YDM,430H31930,Textile/Cloth Tape 19mm B,,,,,,,,26693,NaN


In [7]:
def Fill(level : str):
    for idx in range(df_mmsta.shape[0]):
        if df_mmsta.loc[:,'Level'][idx] == level:
            # PN_arr.append(df_mmsta.loc[idx]['Material Description'])
            df_mmsta.loc[idx,'PN'] = df_mmsta.loc[idx]['Material Description']
            idx_last_l = idx
        else :
            df_mmsta.loc[idx,'PN'] = df_mmsta.loc[idx_last_l]['Material Description']
            # PN_arr.append(df_mmsta.loc[idx_last_l]['Material Description'])
    return df_mmsta

In [8]:
df_mmsta = Fill('0')

In [9]:
def delete_YFG_YDM():
    return df_mmsta[~df_mmsta['Material Type'].isin(['YFG', 'YDM'])]
df_mmsta = delete_YFG_YDM()

In [10]:
def delete_cut_tube_Material_desc():
    df_mmsta['Material Description'] = df_mmsta['Material Description'].apply(lambda x : x.lower())
    df_mmsta[df_mmsta['Material Description'].apply(lambda x : 'cut tube' not in x)]
    return df_mmsta

df_mmsta = delete_cut_tube_Material_desc()

C:\Users\user\AppData\Local\Temp\ipykernel_11984\1666646700.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mmsta['Material Description'] = df_mmsta['Material Description'].apply(lambda x : x.lower())


In [11]:
df_mmsta = df_mmsta.reset_index()

In [12]:
def Fill_with_col(level : str, cols,df):
    idx_last_l = None
    for idx in range(df.shape[0]):
        if df.loc[idx,'Level'] == level:
            df.loc[idx,cols[0]] = df.loc[idx,'Material']
            df.loc[idx,cols[1]] = df.loc[idx,'Material Description']
            idx_last_l = idx
           
            
        else :
            if idx_last_l is not None:
                df.loc[idx,cols[0]] = df.loc[idx_last_l,'Material']
                df.loc[idx,cols[1]] = df.loc[idx_last_l,'Material Description']
                
            else :
                df.loc[idx,cols[0]] = 'None'
                df.loc[idx,cols[1]] = 'None'
    return df

df_mmsta = Fill_with_col('*1', ['SN1','DES1'], df_mmsta)

df_mmsta = Fill_with_col('**2', ['SN2','DES2'], df_mmsta)

df_mmsta = Fill_with_col('***3', ['SN3','DES3'], df_mmsta)

In [13]:
if 'index' in df_mmsta.columns:
    df_mmsta = df_mmsta.drop(columns='index')

In [14]:
import xlwings as xw

wb = xw.Book()
sheet_mmsta = wb.sheets['Feuil1']
sheet_mmsta.range("A1").value = [df_mmsta.columns.tolist()] + df_mmsta.values.tolist()


In [15]:
end_time = time.time()

execution_time = end_time - start_time
print(f"Temps d'exécution : {execution_time / 60} minutes")

Temps d'exécution : 3.489455250898997 minutes


In [16]:
# def fill_added_col(df):
#     i = 0
#     for col in cols_to_add:
#         if col.startswith('S') :
#             df = Fill_with_col(levels[i], col, 'Material',df)
#             i = i + 1
#         elif col.startswith('D') :
#             df = Fill_with_col(levels[i - 1], col, 'Material Description',df)
#     return df
# df_mmsta = fill_added_col(df_mmsta)